# Overview

Adds behavior data (trial and block information) to a Frank Lab nwbfile.

Trial and block tables are added as `TimeIntervals` (same as epochs).

**Trial table** contains information: start_time, stop_time, epoch, block, trial_within_block, trial_within_epoch, start_port, end_port, reward, opto_condition, duration, poke_in, poke_out

**Block table** contains information: start_time, stop_time, epoch, block, maze_configuration, pA, pB, pC, start_trial, end_trial,num_trials, task_type

Also adds a table of **hex centroids** (in video pixel coordinates) to the nwbfile so we can assign the rat's position to the hex the rat was in. Hex centroids are added as a `DynamicTable` named `hex_centroids` in the nwbfile's behavior processing module.

This modifies the nwbfile in-place.

## Usage

The following are needed:

`nwb_path`: Path to a Frank Lab nwbfile with statescriptlogs saved as `AssociatedFiles` objects and behavioral event DIOs in the behavior processing module (this is the standard format for all Frank Lab nwbfiles).

`excel_path`: Path to the excel sheet of experimental notes (including maze configurations for each block in column 'barrier location').

`sheet_name`: (Optional) The name of the sheet tab in the excel file (defaults to 'Daily configs and notes_Bandit+' if not specified.)

`hex_centroids_path`: Path to a csv of hex centroids in video pixel coordinates. This can be created using the [position_tracking/Get_Hex_Centroids.ipynb](position_tracking/Get_Hex_Centroids.ipynb) notebook.

`save_type`: `"pickle"` or `"csv"` to save the trial and block dataframes for each epoch as .pkl or .csv files, `"nwb"` to save the trial and block data directly as time intervals in the nwbfile. It is possible to combine save types, e.g. `save_type="pickle,nwb"`.

Additional argument `overwrite` specifies if existing block and trials data in the nwbfile should be overwritten. Applies only to `save_type="nwb"`. Keeping `overwrite=False` is a good protection against rewriting the file over and over (it will write to the file the first time if there is no existing trial and block data, then stop and complain.)

## Conversion Logs

Running the `add_behavioral_data_to_nwb` function will auto-generate a folder named `{session_id}_logs` containing logs recording everything that happened when parsing behavioral data and modifying the nwbfile. Check the `info` logs to track progress and the `warning` logs to make sure that nothing concerning (e.g. mismatched timestamps) happened. Note that warnings such as `Poke in at statescript time {time} may not match trial printed at {time}` may not always be cause for concern- this can happen even when trials are correctly matched, but there was a delay in statescript printing. DIO times (not statescript times) are always used as ground truth- general alignment with statescript times are just used to ensure everything is correctly matched and warn if there is anything that should be double-checked. Check the `debug` logs for complete information, including the exact timing of events.

## Example 
```
nwb_path = 'data/BraveLu20240519_copy.nwb'
excel_path = 'data/BraveLu_experimental_notes.xlsx'
sheet_name = 'Daily configs and notes_Bandit+'
hex_centroids_path = 'data/hex_coordinates.csv'

add_behavioral_data_to_nwb(nwb_path=nwb_path, excel_path=excel_path, sheet_name=sheet_name, hex_centroids_file_path=hex_centroids_path, save_type="nwb", overwrite=True)
```

In [ ]:
from convert_behavior_Frank_lab import add_behavioral_data_to_nwb
import ndx_franklab_novela

nwb_path = 'data/BraveLu20240519_copy.nwb'
excel_path = 'data/BraveLu_experimental_notes.xlsx'
sheet_name = 'Daily configs and notes_Bandit+'
hex_centroids_path = 'position_tracking/hex_coordinates.csv'

# Add trial and block data to the nwb (modifies existing nwb in-place)
add_behavioral_data_to_nwb(nwb_path, excel_path, sheet_name=sheet_name, 
                           hex_centroids_file_path=hex_centroids_path, save_type="nwb", overwrite=True)


/Users/steph/miniforge3/envs/jdb_to_nwb/lib/python3.12/site-packages/hdmf/backends/hdf5/h5tools.py:663: BrokenLinkWarning: Path to Group altered/broken at /acquisition/e-series
  warnings.warn('Path to Group altered/broken at ' + os.path.join(h5obj.name, k), BrokenLinkWarning)
/Users/steph/miniforge3/envs/jdb_to_nwb/lib/python3.12/site-packages/hdmf/backends/hdf5/h5tools.py:663: BrokenLinkWarning: Path to Group altered/broken at /processing/analog
  warnings.warn('Path to Group altered/broken at ' + os.path.join(h5obj.name, k), BrokenLinkWarning)


### Deleting block and trials table from the nwb
If you need to delete the block and trials table from the nwb, run:

```
nwb_path = 'data/BraveLu20240519_copy.nwb'
delete_blocks_and_trials_from_nwb(nwb_path)
```

This function deletes block and trials tables (stored as TimeIntervals) from the nwbfile if they exist. 
This modifies the file in-place. Note that this will not actually reduce the file size due to limitations in the HDF5 format.

In [4]:
from convert_behavior_Frank_lab import delete_blocks_and_trials_from_nwb

# If needed, delete block and trials table from the nwb
delete_blocks_and_trials_from_nwb(nwb_path)

No block table to delete.
No trials table to delete.


### Deleting hex centroids table from the nwb
If you need to delete the hex centroids table from the nwb, run:

```
nwb_path = 'data/BraveLu20240519_copy.nwb'
delete_hex_centroids_from_nwb(nwb_path)
```

This function deletes the hex centroids (stored as a DynamicTable in the behavior processing module) from the nwbfile if it exists. 
This modifies the file in-place. Note that this will not actually reduce the file size due to limitations in the HDF5 format.

In [6]:
from convert_behavior_Frank_lab import delete_hex_centroids_from_nwb

# If needed, delete the hex centroids table from the nwb
delete_hex_centroids_from_nwb(nwb_path)

No hex centroids table to delete.
